In [ ]:
import pandas as pd
import numpy as np
import chess

import itertools

### Loading data and parsing the text file for moves, results and openings

In [10]:
game_list = [line for line in open('GMallboth.pgn')] 

all_moves_list = []
i = 11
while i <= (len(game_list)/200):           ###### NOTE: '/100' only using 1% of the data availible from grandmasters
    all_moves_list.append(game_list[i])
    i += 16
    
    
all_results = []
i = 13
while i <= (len(game_list)/200):
    all_results.append(game_list[i])
    i += 16
    
all_openings = []
i = 7
while i <= (len(game_list)/200):
    all_openings.append(game_list[i])
    i += 16

In [11]:
def get_move_list(move_string): #### input is a string 
    '''
    Takes a move string that was striped from a PGN format, and removes unwanted characters and conserves SAN format.
    
    Input: String of plain text moves in string format
    Output: List of SAN moves
    
    '''
    testing = move_string 
    testing = testing[0:-2]  ### remove last 2 characters = '\n'
    testing = testing.split('. ')
    
    for i in range(len(testing)):
        testing[i] = testing[i].split(' ')
    for i in testing:
        try:
            del(i[2])
        except:
            continue
    del(testing[0])

    simplelist = list(itertools.chain.from_iterable(testing))
    return(simplelist)

In [12]:
%%time
all_san_list = [get_move_list(game) for game in all_moves_list]
print(len(all_san_list))

1378
CPU times: user 173 ms, sys: 16 ms, total: 189 ms
Wall time: 237 ms


In [13]:
def white_results(result_string_list):
    '''
    Reformats and standardizes stripped result text from chess game in PGN/TXT format
    
    Input: list of strings - (results text with \n)
    Output: win, lose, draw in list format
    
    '''
    white_result_list = []
    for i in range(len(result_string_list)):
            result = result_string_list[i][:-1]
            result = result.replace('1-0', 'win')
            result = result.replace('0-1', 'lose')
            result = result.replace('1/2-1/2', 'draw')
            white_result_list.append(result)
    return(white_result_list)

In [14]:
%%time
white_test = white_results(all_results)

CPU times: user 1.28 ms, sys: 66 µs, total: 1.34 ms
Wall time: 1.35 ms


In [15]:
white_win = []
white_lose = []
white_draw = []

for i in range(len(white_test)):
    if white_test[i] == 'win':
        white_win.append(all_san_list[i])
    elif white_test[i] == 'lose':
        white_lose.append(all_san_list[i])
    else:
        white_draw.append(all_san_list[i])
print(len(white_win), len(white_lose), len(white_draw))

460 293 625


In [16]:
from stockfish import Stockfish

stockfish = Stockfish('Desktop/stockfish-11-mac/Mac/stockfish-11-64')

In [17]:
import chess
import chess.engine

engine = chess.engine.SimpleEngine.popen_uci("Desktop/stockfish-11-mac 2/Mac/stockfish-11-64")

#board = chess.Board()
#info = engine.analyse(board, chess.engine.Limit(time=0.1))
#print("Score:", info["score"])
# Score: +95

board = chess.Board()
print(board)
#info = engine.analyse(board, chess.engine.Limit(depth=10))
#print("Score:", info["score"])
# Score: #+1
#print(info['depth'], info['nodes'], info['time'])
engine.quit()

r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B N R


In [18]:
def clean_fen(string):
    '''
    Takes a fen with extraneous features
    
    input: plain FEN notation i.e: rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR
    output: 1x64 list - with piece abbriviation in position, open space as '1'
    '''
    string = string.replace('8','1'*8)
    string = string.replace('7','1'*7)
    string = string.replace('6','1'*6)
    string = string.replace('5','1'*5)
    string = string.replace('4','1'*4)
    string = string.replace('3','1'*3)
    string = string.replace('2','1'*2)
    string = string.replace('1','1'*1)
    string = string.replace('/','')
    string_list = [i for i in string]
    return(string_list)

def get_bitwise(board_state):
    '''
    Takes a 
    '''
    bs = board_state #test ---- #First posiiton tuple  #winning_position[0][0] - for i in range:
    r_ray = np.zeros(64)
    n_ray = np.zeros(64)
    b_ray = np.zeros(64)
    q_ray = np.zeros(64)
    k_ray = np.zeros(64)
    p_ray = np.zeros(64)
    for i in range(64):
        if bs[i] == 'r':
            r_ray[i] = -1
        if bs[i] == 'R':
            r_ray[i] = 1 
        
        if bs[i] == 'b':
            b_ray[i] = -1
        if bs[i] == 'B':
            b_ray[i] = 1
        
        if bs[i] == 'n':
            n_ray[i] = -1
        if bs[i] == 'N':
            n_ray[i] = 1 
    
        if bs[i] == 'q':
            q_ray[i] = -1
        if bs[i] == 'Q':
            q_ray[i] = 1
    
        if bs[i] == 'k':
            k_ray[i] = -1
        if bs[i] == 'K':
            k_ray[i] = 1 
        
        if bs[i] == 'p':
            p_ray[i] = -1
        if bs[i] == 'P':
            p_ray[i] = 1 
    master_ray = np.array((r_ray, n_ray, b_ray, q_ray, k_ray, p_ray))
    return(master_ray)

In [19]:
len(all_san_list)

1378

In [20]:
test_games = all_san_list


In [21]:
%%time
list_scores = []
engine = chess.engine.SimpleEngine.popen_uci("Desktop/stockfish-11-mac 2/Mac/stockfish-11-64")
for game in test_games:
    board = chess.Board()
    for i in game:
        info = engine.analyse(board, chess.engine.Limit(depth=10))
        x = str(info['score']).replace('#+', '1000')
        x = x.replace('#-', '-1000')
        list_scores.append((int(x), get_bitwise(clean_fen(board.board_fen())))) ### NEED TO GET BITWISE IN HERE
        board.push_san(i)
        
### . chess.engine.PlayResult()

CPU times: user 10min 41s, sys: 20.3 s, total: 11min 1s
Wall time: 28min 41s


In [22]:
df = pd.DataFrame(list_scores)
df

,0,1
0,105,"[[-1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.0, 0...."
1,16,"[[-1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.0, 0...."
2,101,"[[-1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.0, 0...."
3,60,"[[-1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.0, 0...."
4,29,"[[-1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.0, 0...."
...,...,...
122106,723,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
122107,-1362,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
122108,1403,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
122109,-1446,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."


In [ ]:
duck = []
for i in range(len(list_scores)):
    try:
        if i % 2:
            duck.append((int(str(list_scores[i]))/100, 'white'))
        else:
            duck.append((int(str(list_scores[i]))/100, 'black'))
    except:
        continue

In [ ]:
duck

In [ ]:
engine = chess.engine.SimpleEngine.popen_uci("Desktop/stockfish-11-mac 2/Mac/stockfish-11-64")

In [ ]:
engine

In [ ]:
board = chess.Board()
board.push_san('d4')
print(board)
